In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from tqdm import tqdm  # for progress bar, optional


In [ ]:
import cv2
import numpy as np
import os
from functools import reduce
from tqdm import tqdm  # for progress bar, optional
import matplotlib.pyplot as plt

def smart_leaf_grabcut(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return np.zeros((100, 100, 3), dtype=np.uint8)  # Return blank

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    masks = []
    masks.append(cv2.inRange(hsv, (25, 40, 40), (85, 255, 255)))    # Green
    masks.append(cv2.inRange(hsv, (15, 50, 50), (35, 255, 255)))    # Yellow
    masks.append(cv2.inRange(hsv, (0, 50, 20), (20, 255, 200)))     # Brownish
    masks.append(cv2.inRange(hsv, (0, 60, 60), (10, 255, 255)))     # Reddish

    leaf_mask = reduce(cv2.bitwise_or, masks)  
    bgModel = np.zeros((1, 65), np.float64)
    fgModel = np.zeros((1, 65), np.float64)

    if np.count_nonzero(leaf_mask) < 100:
        print(f"[{os.path.basename(image_path)}] Mask too weak, using rectangle fallback.")
        grabcut_mask = np.zeros(img.shape[:2], dtype=np.uint8)
        rect = (10, 10, img.shape[1] - 20, img.shape[0] - 20)
        cv2.grabCut(img, grabcut_mask, rect, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)
    else:
        grabcut_mask = np.full(img.shape[:2], cv2.GC_PR_BGD, dtype=np.uint8)
        grabcut_mask[leaf_mask > 0] = cv2.GC_PR_FGD
        h, w = grabcut_mask.shape
        grabcut_mask[h//3:2*h//3, w//3:2*w//3] = cv2.GC_FGD
        cv2.grabCut(img, grabcut_mask, None, bgModel, fgModel, 5, cv2.GC_INIT_WITH_MASK)

    final_mask = np.where((grabcut_mask == cv2.GC_FGD) | (grabcut_mask == cv2.GC_PR_FGD), 1, 0).astype(np.uint8)
    result = img_rgb * final_mask[:, :, np.newaxis]

    return result

def process_dataset(input_root, output_root):
    for root, dirs, files in os.walk(input_root):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                relative_path = os.path.relpath(img_path, input_root)
                save_path = os.path.join(output_root, relative_path)
                save_dir = os.path.dirname(save_path)
                os.makedirs(save_dir, exist_ok=True)

                segmented = smart_leaf_grabcut(img_path)
                segmented_bgr = cv2.cvtColor(segmented, cv2.COLOR_RGB2BGR)
                cv2.imwrite(save_path, segmented_bgr)

# Paths
dataset_base = "New Plant Diseases Dataset(Augmented)"
output_base = "SegmentedDataset"

for split in ["train", "test", "valid"]:
    input_folder = os.path.join(dataset_base, split)
    output_folder = os.path.join(output_base, split)
    print(f"Processing {split} folder...")
    process_dataset(input_folder, output_folder)

print("All images processed and saved.")


Processing train folder...
